In [1]:
import numpy as np 
import urdf2casadi.urdfparser as u2c
from urdf2casadi.geometry import plucker
from urdf_parser_py.urdf import URDF, Pose
import PyKDL as kdl
import kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl
import pybullet as pb


In [2]:
def median(lst):
    n = len(lst)
    if n < 1:
            return None
    if n % 2 == 1:
            return sorted(lst)[n//2]
    else:
            return sum(sorted(lst)[n//2-1:n//2+1])/2.0
        
def average(lst): 
    return sum(lst) / len(lst) 


def c_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToCoriolis(q_kdl, qdot_kdl, g_kdl)


def c_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    c_u2c(q_none, qdot_none)
    
def c_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    c_pb = pb.calculateInverseDynamics(pbmodel, q_none, qdot_none, qddot)
    
    
def c_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.NonlinearEffects(rbdlmodel, q_np, qdot_np, c_rbdl)


def g_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToGravity(q_kdl, g_kdl)


def g_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_u2c(q_none)
    
def g_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_pb = pb.calculateInverseDynamics(pbmodel, q_none, qdot, qddot)
    
def g_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, qddot_np, g_rbdl)
    
    
def id_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    id_u2c(q_none, qdot_none, qddot_none)
    
    
def id_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2 
    id_pb = pb.calculateInverseDynamics(pbmodel, q_none, qdot_none, qddot_none)
    
    
def id_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, qddot_np, id_rbdl)

    
def m_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToMass(q_kdl, m_kdl)


def m_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    m_u2c(q_none)
    
def m_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    m = pb.calculateMassMatrix(pbmodel, q_none)
    
    
def m_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.CompositeRigidBodyAlgorithm(rbdlmodel, q_np, m_rbdl)
    
    
def aba_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_aba_u2c(q_none, qdot_none, tau_none)
    
    
def crba_u2c_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_crba_u2c(q_none, qdot_none, tau_none)
    
    
def fd_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.ForwardDynamics(rbdlmodel, q_np, qdot_np, tau_np, qddot_np)
    

In [3]:
#pbmodel = pb.loadURDF("pantilt.urdf")
sim = pb.connect(pb.DIRECT)
#pbmodel = pb.loadURDF("/urdf4timing/1dof.urdf")
ndofs = 10
urdf_nr = list(range(ndofs + 1))
urdf_nr.pop(0)


nitr = 1000
nrepeat = 100
    

path_to_urdf = "/home/lmjohann/urdf2casadi/examples/urdf/snake_robot.urdf"
root = 'base_link'
tip = "link16"

ok, kdl_tree = kdlurdf.treeFromFile(path_to_urdf)
kdlmodel = kdl_tree.getChain(root,tip)

rbdlmodel = rbdl.loadModel(path_to_urdf)
pbmodel = pb.loadURDF(path_to_urdf, useFixedBase=True, flags = pb.URDF_USE_INERTIA_FROM_FILE)
asd = u2c.URDFparser()
robot_desc = asd.from_file(path_to_urdf)

jlist, names, qmax, qmin = asd.get_joint_info(root, tip)
njoints = asd.get_n_joints(root, tip)

gravity_kdl = kdl.Vector()
gravity_kdl[2] = -9.81
gravity_u2c = [0, 0, -9.81]

#rbdl declarations
q_np = np.zeros(njoints)
qdot_np = np.zeros(njoints)
qddot_np = np.zeros(njoints)
g_rbdl = np.zeros(njoints)
c_rbdl = np.zeros(njoints)
id_rbdl = np.zeros(njoints)

#kdl declatations
q_kdl = kdl.JntArray(njoints)
qdot_kdl = kdl.JntArray(njoints)
g_kdl = kdl.JntArray(njoints)

#u2c and pybullet declarations
q_none = [None]*njoints
qdot_none = [None]*njoints
qddot_none = [None]*njoints
g_pb = [None]*njoints

qddot = []
for i in range(njoints):
    qddot.append(0.)

qdot = []
for i in range(njoints):
    qdot.append(0.)
    
    
m_kdl = kdl.JntSpaceInertiaMatrix(njoints)
m_rbdl = (rbdlmodel.q_size, rbdlmodel.q_size)
m_rbdl = np.zeros(m_rbdl)
tau_np = np.zeros(njoints)
tau_none = [None]*njoints

g_u2c = asd.get_gravity_rnea(root, tip, gravity_u2c)
c_u2c = asd.get_coriolis_rnea(root, tip)
m_u2c = asd.get_inertia_matrix_crba(root, tip)
fd_aba_u2c = asd.get_forward_dynamics_aba(root, tip, gravity_u2c)
fd_crba_u2c = asd.get_forward_dynamics_crba(root, tip, gravity_u2c)
id_u2c = asd.get_inverse_dynamics_rnea(root, tip, gravity_u2c)




/home/lmjohann/urdf2casadi/examples/urdf/snake_robot.urdf


In [4]:
timeit_g_u2c = repeat("g_u2c_func()", setup = "from __main__ import g_u2c_func", repeat = nrepeat, number = nitr)
mediantime_g_u2c  = median(timeit_g_u2c)/nitr*1000000

timeit_g_pb = repeat("g_pb_func()", setup = "from __main__ import g_pb_func", repeat = nrepeat, number = nitr)
mediantime_g_pb = median(timeit_g_pb)/nitr*1000000

timeit_g_rbdl = repeat("g_rbdl_func()", setup = "from __main__ import g_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_g_rbdl = median(timeit_g_rbdl)/nitr*1000000

timeit_g_kdl = repeat("g_kdl_func()", setup = "from __main__ import g_kdl_func", repeat = nrepeat, number = nitr)
mediantime_g_kdl = median(timeit_g_kdl)/nitr*1000000


timeit_c_u2c = repeat("c_u2c_func()", setup = "from __main__ import c_u2c_func", repeat = nrepeat, number = nitr)
mediantime_c_u2c  = median(timeit_c_u2c)/nitr*1000000

timeit_c_pb = repeat("c_pb_func()", setup = "from __main__ import c_pb_func", repeat = nrepeat, number = nitr)
mediantime_c_pb = median(timeit_c_pb)/nitr*1000000

timeit_c_rbdl = repeat("c_rbdl_func()", setup = "from __main__ import c_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_c_rbdl = median(timeit_c_rbdl)/nitr*1000000

timeit_c_kdl = repeat("c_kdl_func()", setup = "from __main__ import c_kdl_func", repeat = nrepeat, number = nitr)
mediantime_c_kdl = median(timeit_c_kdl)/nitr*1000000

timeit_id_u2c = repeat("id_u2c_func()", setup = "from __main__ import id_u2c_func", repeat = nrepeat, number = nitr)
mediantime_id_u2c  = median(timeit_id_u2c)/nitr*1000000

timeit_id_pb = repeat("id_pb_func()", setup = "from __main__ import id_pb_func", repeat = nrepeat, number = nitr)
mediantime_id_pb = median(timeit_id_pb)/nitr*1000000

timeit_id_rbdl = repeat("id_rbdl_func()", setup = "from __main__ import id_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_id_rbdl = median(timeit_id_rbdl)/nitr*1000000


timeit_m_u2c = repeat("m_u2c_func()", setup = "from __main__ import m_u2c_func", repeat = nrepeat, number = nitr)
mediantime_m_u2c  = median(timeit_m_u2c)/nitr*1000000

timeit_m_pb = repeat("m_pb_func()", setup = "from __main__ import m_pb_func", repeat = nrepeat, number = nitr)
mediantime_m_pb = median(timeit_m_pb)/nitr*1000000

timeit_m_rbdl = repeat("m_rbdl_func()", setup = "from __main__ import m_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_m_rbdl = median(timeit_m_rbdl)/nitr*1000000

timeit_m_kdl = repeat("m_kdl_func()", setup = "from __main__ import m_kdl_func", repeat = nrepeat, number = nitr)
mediantime_m_kdl = median(timeit_m_kdl)/nitr*1000000


timeit_fdaba_u2c = repeat("aba_u2c_func()", setup = "from __main__ import aba_u2c_func", repeat = nrepeat, number = nitr)
mediantime_fdaba_u2c = median(timeit_fdaba_u2c)/nitr*1000000

timeit_fdcrba_u2c = repeat("crba_u2c_func()", setup = "from __main__ import crba_u2c_func", repeat = nrepeat, number = nitr)
mediantime_fdcrba_u2c = median(timeit_fdcrba_u2c)/nitr*1000000

timeit_fd_rbdl = repeat("fd_rbdl_func()", setup = "from __main__ import fd_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_fd_rbdl = median(timeit_fd_rbdl)/nitr*1000000


In [5]:
print "\nsnake_g_u2c=", mediantime_g_u2c
print "\nsnake_id_u2c=", mediantime_id_u2c
print "\nsnake_c_u2c=", mediantime_c_u2c
print "\nsnake_m_u2c=", mediantime_m_u2c

print "\nsnake_g_rbdl=", mediantime_g_rbdl
print "\nsnake_id_rbdl=", mediantime_id_rbdl
print "\nsnake_c_rbdl=", mediantime_c_rbdl
print "\nsnake_m_rbdl=", mediantime_m_rbdl

print "\nsnake_g_pb=", mediantime_g_pb
print "\nsnake_id_pb=", mediantime_id_pb
print "\nsnake_c_pb=", mediantime_c_pb
print "\nsnake_m_pb=", mediantime_m_pb

print "\nsnake_g_kdl=", mediantime_g_kdl
print "\nsnake_c_kdl=", mediantime_c_kdl
print "\nsnake_m_kdl=", mediantime_m_kdl

print "\nfdrbdl=", mediantime_fd_rbdl
print "\nfdcrba=", mediantime_fdcrba_u2c 
print "\nfdaba=", mediantime_fdaba_u2c


snake_g_u2c= 21.6684341431

snake_id_u2c= 52.229642868

snake_c_u2c= 37.2015237808

snake_m_u2c= 22.8976011276

snake_g_rbdl= 13.7885808945

snake_id_rbdl= 28.7569761276

snake_c_rbdl= 20.7204818726

snake_m_rbdl= 13.7399435043

snake_g_pb= 11.3579034805

snake_id_pb= 25.0895023346

snake_c_pb= 18.1304216385

snake_m_pb= 16.1025524139

snake_g_kdl= 26.0291099548

snake_c_kdl= 36.7465019226

snake_m_kdl= 28.9385318756

fdrbdl= 31.9910049438

fdcrba= 68.0139064789

fdaba= 53.0254840851


In [6]:

snake_g_u2c= 22.607922554

snake_id_u2c= 53.5010099411

snake_C_u2c= 36.9205474854

snake_M_u2c= 22.8043794632

snake_g_rbdl= 14.088511467

snake_id_rbdl= 29.7459363937

snake_C_rbdl= 21.2104320526

snake_M_rbdl= 14.1924619675

snake_g_pb= 11.3346576691

snake_id_pb= 25.2009630203

snake_C_pb= 18.262386322

snake_M_pb= 15.8640146255

snake_g_kdl= 22.5094556808

snake_C_kdl= 31.4583778381

snake_M_kdl= 25.65741539
fdrbdl= 33.0110788345

fdcrba= 67.3921108246

fdaba= 54.6615123749



snake_g_u2c= 21.6684341431

snake_id_u2c= 52.229642868

snake_c_u2c= 37.2015237808

snake_m_u2c= 22.8976011276

snake_g_rbdl= 13.7885808945

snake_id_rbdl= 28.7569761276

snake_c_rbdl= 20.7204818726

snake_m_rbdl= 13.7399435043

snake_g_pb= 11.3579034805

snake_id_pb= 25.0895023346

snake_c_pb= 18.1304216385

snake_m_pb= 16.1025524139

snake_g_kdl= 26.0291099548

snake_c_kdl= 36.7465019226

snake_m_kdl= 28.9385318756

fdrbdl= 31.9910049438

fdcrba= 68.0139064789

fdaba= 53.0254840851


# Derivatives 

## q

In [7]:
import casadi as cs

qs =cs.SX.sym("qs",njoints)
qsdot =cs.SX.sym("qsdot",njoints)
qsddot =cs.SX.sym("qsddot",njoints)
taus =cs.SX.sym("taus",njoints)

c_dq = cs.jacobian(c_u2c(qs, qsdot), qs)
g_dq = cs.jacobian(g_u2c(qs), qs)
m_dq = cs.jacobian(m_u2c(qs), qs)
id_dq = cs.jacobian(id_u2c(qs, qsdot, qsddot), qs)
fd_aba_dq = cs.jacobian(fd_aba_u2c(qs, qsdot, taus), qs)
fd_crba_dq = cs.jacobian(fd_crba_u2c(qs, qsdot, taus), qs)


g_dq = cs.Function("g_dq", [qs], [g_dq], {"jit": True, "jit_options":{"flags":"-Ofast"}})
c_dq = cs.Function("C_dq", [qs, qsdot], [c_dq], {"jit": True, "jit_options":{"flags":"-Ofast"}})
m_dq = cs.Function("M_dq", [qs], [m_dq], {"jit": True, "jit_options":{"flags":"-Ofast"}})
id_dq = cs.Function("id_dq", [qs, qsdot, qsddot], [id_dq], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_aba_dq = cs.Function("fd_aba_dq", [qs, qsdot, taus], [fd_aba_dq], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_crba_dq = cs.Function("fd_crba_dq", [qs, qsdot, taus], [fd_crba_dq], {"jit": True, "jit_options":{"flags":"-Ofast"}})

In [8]:
def fdcrba_dq_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_crba_dq(q_none, qdot_none, tau_none)

def fdaba_dq_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_aba_dq(q_none, qdot_none, tau_none)
    
    
def m_dq_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2   
    m_dq(q_none)
    
def g_dq_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_dq(q_none)


def c_dq_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    c_dq(q_none, qdot_none)
    
def id_dq_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    id_dq(q_none, qdot_none, qddot_none)

def aba_dq_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    fd_aba_dq(q_none, qdot_none, tau_none)
    
def crba_dq_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    
    fd_crba_dq(q_none, qdot_none, tau_none)

In [9]:
timeit_idj = repeat("id_dq_func()", setup = "from __main__ import id_dq_func", repeat = nrepeat, number = nitr)
mediantime_idj = median(timeit_idj)/nitr*1000000

timeit_gj = repeat("g_dq_func()", setup = "from __main__ import g_dq_func", repeat = nrepeat, number = nitr)
mediantime_gj = median(timeit_gj)/nitr*1000000

timeit_cj = repeat("c_dq_func()", setup = "from __main__ import c_dq_func", repeat = nrepeat, number = nitr)
mediantime_cj = median(timeit_cj)/nitr*1000000

timeit_mj = repeat("m_dq_func()", setup = "from __main__ import m_dq_func", repeat = nrepeat, number = nitr)
mediantime_mj = median(timeit_mj)/nitr*1000000

timeit_fdabaj = repeat("aba_dq_func()", setup = "from __main__ import aba_dq_func", repeat = nrepeat, number = nitr)
mediantime_fd_abaj = median(timeit_fdabaj)/nitr*1000000

timeit_fdcrbaj = repeat("crba_dq_func()", setup = "from __main__ import crba_dq_func", repeat = nrepeat, number = nitr)
mediantime_fd_crbaj = median(timeit_fdcrbaj)/nitr*1000000

   
print "\nsnake_id_d=", mediantime_idj
print "\nsnake_g_d=", mediantime_gj
print "\nsnake_c_d=", mediantime_cj
print "\nsnake_m_d=", mediantime_mj
print "\nsnake_fdaba_d=", mediantime_fd_abaj
print "\nsnake_fdcrba_d=", mediantime_fd_crbaj


snake_id_d= 57.6125383377

snake_g_d= 23.4861373901

snake_c_d= 44.7905063629

snake_m_d= 34.6865653992

snake_fdaba_d= 71.0815191269

snake_fdcrba_d= 276.358485222




snake_id_d= 58.5190057755

snake_g_d= 24.9165296555

snake_C_d= 43.6835289001

snake_M_d= 38.4865999222


snake_m_d= 34.544467926

snake_fdaba_d= 63.8264417648

snake_fdcrba_d= 273.746490479



snake_id_d= 57.6125383377

snake_g_d= 23.4861373901

snake_c_d= 44.7905063629

snake_m_d= 34.6865653992

snake_fdaba_d= 71.0815191269

snake_fdcrba_d= 276.358485222

## qdot 

In [10]:
c_dqdot = cs.jacobian(c_u2c(qs, qsdot), qs)
id_dqdot = cs.jacobian(id_u2c(qs, qsdot, qsddot), qsdot)
fd_aba_dqdot = cs.jacobian(fd_aba_u2c(qs, qsdot, taus), qsdot)
fd_crba_dqdot = cs.jacobian(fd_crba_u2c(qs, qsdot, taus), qsdot)

c_dqdot = cs.Function("c_dq", [qs, qsdot], [c_dqdot], {"jit": True, "jit_options":{"flags":"-Ofast"}})
id_dqdot = cs.Function("id_dq", [qs, qsdot, qsddot], [id_dqdot], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_aba_dqdot = cs.Function("fd_aba_dq", [qs, qsdot, taus], [fd_aba_dqdot], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_crba_dqdot = cs.Function("fd_crba_dq", [qs, qsdot, taus], [fd_crba_dqdot], {"jit": True, "jit_options":{"flags":"-Ofast"}})

In [11]:


def c_dqdot_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    c_dqdot(q_none, qdot_none)
    
def id_dqdot_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    id_dqdot(q_none, qdot_none, qddot_none)

def aba_dqdot_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    fd_aba_dqdot(q_none, qdot_none, tau_none)
    
def crba_dqdot_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    
    fd_crba_dqdot(q_none, qdot_none, tau_none)

In [12]:
#timeit_idj = repeat("id_dqdot_func()", setup = "from __main__ import id_dqdot_func", repeat = nrepeat, number = nitr)
mediantime_idj = median(timeit_idj)/nitr*1000000

#timeit_cj = repeat("c_dqdot_func()", setup = "from __main__ import c_dqdot_func", repeat = nrepeat, number = nitr)
mediantime_cj = median(timeit_cj)/nitr*1000000


timeit_fdabaj = repeat("aba_dqdot_func()", setup = "from __main__ import aba_dqdot_func", repeat = nrepeat, number = nitr)
mediantime_fd_abaj = median(timeit_fdabaj)/nitr*1000000

timeit_fdcrbaj = repeat("crba_dqdot_func()", setup = "from __main__ import crba_dqdot_func", repeat = nrepeat, number = nitr)
mediantime_fd_crbaj = median(timeit_fdcrbaj)/nitr*1000000

   
print "\nsnake_id_dqdot=", mediantime_idj
print "\nsnake_C_dqdot=", mediantime_cj
print "\nsnake_fdaba_dqdot=", mediantime_fd_abaj
print "\nsnake_fdcrba_dqdot=", mediantime_fd_crbaj


snake_id_dqdot= 57.6125383377

snake_C_dqdot= 44.7905063629

snake_fdaba_dqdot= 61.6301298141

snake_fdcrba_dqdot= 73.3659267426


In [13]:

snake_id_d= 53.680062294

snake_C_d= 36.3329648972

snake_fdaba_d= 57.6950311661

snake_fdcrba_d= 73.1304883957

## qddot / tau

In [14]:


id_dqddot = cs.jacobian(id_u2c(qs, qsdot, qsddot), qsddot)
fd_aba_dtau = cs.jacobian(fd_aba_u2c(qs, qsdot, taus), taus)
fd_crba_dtau = cs.jacobian(fd_crba_u2c(qs, qsdot, taus), taus)

id_dqddot = cs.Function("id_qddot", [qs, qsdot, qsddot], [id_dqddot], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_aba_dtau = cs.Function("fd_aba_dtau", [qs, qsdot, taus], [fd_aba_dtau], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_crba_dtau = cs.Function("fd_crba_dtau", [qs, qsdot, taus], [fd_crba_dtau], {"jit": True, "jit_options":{"flags":"-Ofast"}})

In [15]:
def id_dqddot_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    id_dqddot(q_none, qdot_none, qddot_none)

def aba_dtau_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    fd_aba_dtau(q_none, qdot_none, tau_none)
    
def crba_dtau_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    
    fd_crba_dtau(q_none, qdot_none, tau_none)

In [16]:
timeit_idj = repeat("id_dqddot_func()", setup = "from __main__ import id_dqddot_func", repeat = nrepeat, number = nitr)
mediantime_idj = median(timeit_idj)/nitr*1000000

timeit_fdabaj = repeat("aba_dtau_func()", setup = "from __main__ import aba_dtau_func", repeat = nrepeat, number = nitr)
mediantime_fd_abaj = median(timeit_fdabaj)/nitr*1000000

timeit_fdcrbaj = repeat("crba_dtau_func()", setup = "from __main__ import crba_dtau_func", repeat = nrepeat, number = nitr)
mediantime_fd_crbaj = median(timeit_fdcrbaj)/nitr*1000000

   
print "\nsnake_id_d=", mediantime_idj
print "\nsnake_fdaba_d=", mediantime_fd_abaj
print "\nsnake_fdcrba_d=", mediantime_fd_crbaj


snake_id_d= 57.4533939362

snake_fdaba_d= 59.4005584717

snake_fdcrba_d= 65.9115314484


In [17]:

snake_id_d= 54.7025203705

snake_fdaba_d= 58.310508728

snake_fdcrba_d= 65.9115314484

## Derivatives all

In [4]:
import casadi as cs
qs =cs.SX.sym("qs",njoints)
qsdot =cs.SX.sym("qsdot",njoints)
qsddot =cs.SX.sym("qsddot",njoints)
qsdddot = cs.SX.sym("qsdddot", njoints)
taus =cs.SX.sym("taus",njoints)
tausdot =cs.SX.sym("tausdot",njoints)

c_vec = cs.vertcat(qs, qsdot)
c_dvec = cs.vertcat(qsdot, qsddot)
id_vec = cs.vertcat(qs, qsdot, qsddot)
id_dvec = cs.vertcat(qsdot, qsdot, qsdddot)
fd_vec = cs.vertcat(qs, qsdot, qsddot)
fd_dvec = cs.vertcat(qsdot, qsdot, tausdot)

dc = cs.jacobian(c_u2c(qs, qsdot), c_vec)
dc = cs.Function("dc", [qs, qsdot], [dc], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dctimes = cs.jtimes(c_u2c(qs, qsdot), c_vec, c_dvec)
dctimes = cs.Function("dctimes", [qs, qsdot, qsddot], [dctimes], {"jit": True, "jit_options":{"flags":"-Ofast"}})

did = cs.jacobian(id_u2c(qs, qsdot, qsddot), id_vec)
did = cs.Function("did", [qs, qsdot, qsddot], [did], {"jit": True, "jit_options":{"flags":"-Ofast"}})

didtimes = cs.jtimes(id_u2c(qs, qsdot, qsddot), id_vec, id_dvec)
didtimes = cs.Function("didtimes", [qs, qsdot, qsddot, qsdddot], [didtimes], {"jit": True, "jit_options":{"flags":"-Ofast"}})


dfd_aba = cs.jacobian(fd_aba_u2c(qs, qsdot, taus), fd_vec)
dfd_aba = cs.Function("dfd_aba", [qs, qsdot, taus], [dfd_aba], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dfdtimes_aba = cs.jtimes(fd_aba_u2c(qs, qsdot, taus), fd_vec, fd_dvec)
dfdtimes_aba = cs.Function("dfdtimes_aba", [qs, qsdot, taus, qsddot, tausdot], [dfdtimes_aba], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dfd_crba = cs.jacobian(fd_crba_u2c(qs, qsdot, taus), fd_vec)
dfd_crba = cs.Function("dfd_crba", [qs, qsdot, taus], [dfd_crba], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dfdtimes_crba = cs.jtimes(fd_crba_u2c(qs, qsdot, taus), fd_vec, fd_dvec)
dfdtimes_crba = cs.Function("dfdtimes_crba", [qs, qsdot, taus, qsddot, tausdot], [dfdtimes_crba], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dg = cs.jacobian(g_u2c(qs), qs)
dg = cs.Function("dg", [qs], [dg], {"jit": True, "jit_options":{"flags":"-Ofast"}})


dm = cs.jacobian(m_u2c(qs), qs)
dm = cs.Function("dm", [qs], [dm], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dgtimes = cs.jtimes(g_u2c(qs), qs, qsdot)
dgtimes = cs.Function("dgtimes", [qs, qsdot], [dgtimes], {"jit": True, "jit_options":{"flags":"-Ofast"}})

dmtimes = cs.jtimes(m_u2c(qs), qs, qsdot)
dmtimes = cs.Function("dmtimes", [qs, qsdot], [dmtimes], {"jit": True, "jit_options":{"flags":"-Ofast"}})


print "\nsnake_nodes_did = ", did.n_nodes()
print "\nsnake_nodes_dc = ", dc.n_nodes()
print "\nsnake_nodes_dm = ", dm.n_nodes()
print "\nsnake_nodes_dg = ", dg.n_nodes()
print "\nsnake_nodes_dfd_aba = ", dfd_aba.n_nodes()
print "\nsnake_nodes_dfd_crba =", dfd_crba.n_nodes()

print "\nsnake_nodes_didtimes = ", didtimes.n_nodes()
print "\nsnake_nodes_dctimes = ", dctimes.n_nodes()
print "\nsnake_nodes_dmtimes = ", dmtimes.n_nodes()
print "\nsnake_nodes_dgtimes = ", dgtimes.n_nodes()
print "\nsnake_nodes_dfdtimes_aba = ", dfdtimes_aba.n_nodes()
print "\nsnake_nodes_dfdtimes_crba =", dfdtimes_crba.n_nodes()


def dfd_crba_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dfd_crba(q_none, qdot_none, tau_none)
   
def dfdtimes_crba_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        taudot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dfdtimes_crba(q_none, qdot_none, tau_none, qddot_none, taudot_none)

def dfd_aba_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dfd_aba(q_none, qdot_none, tau_none)
   
def dfdtimes_aba_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        taudot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dfdtimes_aba(q_none, qdot_none, tau_none, qddot_none, taudot_none)
   
   
   
def dm_func():  
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2  
    dm(q_none)
   
def dg_func():  
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dg(q_none)
   
   
def dmtimes_func():  
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2  
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dmtimes(q_none, qdot_none)
   
def dgtimes_func():  
    for j in range(njoints):
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2        
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    dgtimes(q_none, qdot_none)


def dc_func():  
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    dc(q_none, qdot_none)
   
   
def dctimes_func():  
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    dctimes(q_none, qdot_none, qddot_none)
   
   
def did_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    did(q_none, qdot_none, qddot_none)
   
def didtimes_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    didtimes(q_none, qdot_none, qddot_none, qdddot_none)

qdddot_none = [None]*njoints
taudot_none = [None]*njoints





ur5_nodes_did =  16159

ur5_nodes_dc =  11792

ur5_nodes_dm =  11635

ur5_nodes_dg =  2347

ur5_nodes_dfd_aba =  40901

ur5_nodes_dfd_crba = 431178

ur5_nodes_didtimes =  2029

ur5_nodes_dctimes =  1660

ur5_nodes_dmtimes =  3823

ur5_nodes_dgtimes =  687

ur5_nodes_dfdtimes_aba =  7218

ur5_nodes_dfdtimes_crba = 44940


In [ ]:

snake_nodes_did =  16159

snake_nodes_dc =  11792

snake_nodes_dm =  11635

snake_nodes_dg =  2347

snake_nodes_dfd_aba =  40901

snake_nodes_dfd_crba = 431178

snake_nodes_didtimes =  2029

snake_nodes_dctimes =  1660

snake_nodes_dmtimes =  3823

snake_nodes_dgtimes =  687

snake_nodes_dfdtimes_aba =  7218

snake_nodes_dfdtimes_crba = 44940


In [5]:
timeit_did = repeat("did_func()", setup = "from __main__ import did_func", repeat = nrepeat, number = nitr)
mediantime_did = median(timeit_did)/nitr*1000000

timeit_dg = repeat("dg_func()", setup = "from __main__ import dg_func", repeat = nrepeat, number = nitr)
mediantime_dg = median(timeit_dg)/nitr*1000000

timeit_dc = repeat("dc_func()", setup = "from __main__ import dc_func", repeat = nrepeat, number = nitr)
mediantime_dc = median(timeit_dc)/nitr*1000000

timeit_dm = repeat("dm_func()", setup = "from __main__ import dm_func", repeat = nrepeat, number = nitr)
mediantime_dm = median(timeit_dm)/nitr*1000000

timeit_dfd_aba = repeat("dfd_aba_func()", setup = "from __main__ import dfd_aba_func", repeat = nrepeat, number = nitr)
mediantime_dfd_aba = median(timeit_dfd_aba)/nitr*1000000

timeit_dfd_crba = repeat("dfd_crba_func()", setup = "from __main__ import dfd_crba_func", repeat = nrepeat, number = nitr)
mediantime_dfd_crba = median(timeit_dfd_crba)/nitr*1000000

   
print "\nsnake_did =", mediantime_did
print "\nsnake_dg =", mediantime_dg
print "\nsnake_dc =", mediantime_dc
print "\nsnake_dm =", mediantime_dm
print "\nsnake_dfd_aba =", mediantime_dfd_aba
print "\nsnake_dfd_crba =", mediantime_dfd_crba


ur5_did = 67.6834583282

ur5_dg = 23.8734483719

ur5_dc = 46.7829704285

ur5_dm = 36.7213487625

ur5_dfd_aba = 82.4838876724

ur5_dfd_crba = 286.731004715


snake_did = 67.6834583282

snake_dg = 23.8734483719

snake_dc = 46.7829704285

snake_dm = 36.7213487625

snake_dfd_aba = 82.4838876724

snake_dfd_crba = 286.731004715

In [6]:
timeit_didtimes = repeat("didtimes_func()", setup = "from __main__ import didtimes_func", repeat = nrepeat, number = nitr)
mediantime_didtimes = median(timeit_didtimes)/nitr*1000000

timeit_dgtimes = repeat("dgtimes_func()", setup = "from __main__ import dgtimes_func", repeat = nrepeat, number = nitr)
mediantime_dgtimes = median(timeit_dgtimes)/nitr*1000000

timeit_dctimes = repeat("dctimes_func()", setup = "from __main__ import dctimes_func", repeat = nrepeat, number = nitr)
mediantime_dctimes = median(timeit_dctimes)/nitr*1000000

timeit_dmtimes = repeat("dmtimes_func()", setup = "from __main__ import dmtimes_func", repeat = nrepeat, number = nitr)
mediantime_dmtimes = median(timeit_dmtimes)/nitr*1000000

timeit_dfdtimes_aba = repeat("dfdtimes_aba_func()", setup = "from __main__ import dfdtimes_aba_func", repeat = nrepeat, number = nitr)
mediantime_dfdtimes_aba = median(timeit_dfdtimes_aba)/nitr*1000000

timeit_dfdtimes_crba = repeat("dfdtimes_crba_func()", setup = "from __main__ import dfdtimes_crba_func", repeat = nrepeat, number = nitr)
mediantime_dfdtimes_crba = median(timeit_dfdtimes_crba)/nitr*1000000

print "\nsnake_didtimes =", mediantime_didtimes
print "\nsnake_dgtimes =", mediantime_dgtimes
print "\nsnake_dctimes =", mediantime_dctimes
print "\nsnake_dmtimes =", mediantime_dmtimes
print "\nsnake_dfdtimes_aba =", mediantime_dfdtimes_aba
print "\nsnake_dfdtimes_crba =", mediantime_dfdtimes_crba


ur5_didtimes = 71.1984634399

ur5_dgtimes = 38.4654998779

ur5_dctimes = 54.9699068069

ur5_dmtimes = 40.619969368

ur5_dfdtimes_aba = 81.8629264832

ur5_dfdtimes_crba = 127.549052238


snake_didtimes = 71.1984634399

snake_dgtimes = 38.4654998779

snake_dctimes = 54.9699068069

snake_dmtimes = 40.619969368

snake_dfdtimes_aba = 81.8629264832

snake_dfdtimes_crba = 127.549052238

# Numebr of Operations

In [18]:
print "\nsnake_nodes_id=", id_u2c.n_nodes()
print "\nsnake_nodes_c=", c_u2c.n_nodes()
print "\nsnake_nodes_m=", m_u2c.n_nodes()
print "\nsnake_nodes_g=", g_u2c.n_nodes()
print "\nsnake_nodes_fdaba", fd_aba_u2c.n_nodes()
print "\nsnake_nodes_fdcrba=", fd_crba_u2c.n_nodes() 

print "\n\nsnake_nodes_id_dq=", id_dq.n_nodes()
print "\nsnake_nodes_c_dq=", c_dq.n_nodes()
print "\nsnake_nodes_m_dq =", m_dq.n_nodes()
print "\nsnake_nodes_g_dq =", g_dq.n_nodes()
print "\nsnake_nodes_fdaba_dq =", fd_aba_dq.n_nodes()
print "\nsnake_nodes_fdcrba_dq =", fd_crba_dq.n_nodes()     

print "\n\nsnake_nodes_id_dqdot=", id_dqdot.n_nodes()
print "\nsnake_nodes_c_dqdot =", c_dqdot.n_nodes()
print "\nsnake_nodes_fdaba_dqdot =", fd_aba_dqdot.n_nodes()
print "\nsnake_nodes_fdcrba_dqdot =", fd_crba_dqdot.n_nodes()     

print "\n\nsnake_nodes_id_dqddot=", id_dqddot.n_nodes()
print "\nsnake_nodes_fdaba_dtau =", fd_aba_dtau.n_nodes()
print "\nsnake_nodes_fdcrba_dtau =", fd_crba_dtau.n_nodes()   


snake_nodes_id= 710

snake_nodes_c= 572

snake_nodes_m= 1217

snake_nodes_g= 224

snake_nodes_fdaba 2272

snake_nodes_fdcrba= 14786


snake_nodes_id_dq= 7128

snake_nodes_c_dq= 6217

snake_nodes_m_dq = 11635

snake_nodes_g_dq = 2347

snake_nodes_fdaba_dq = 30719

snake_nodes_fdcrba_dq = 417474


snake_nodes_id_dqdot= 5755

snake_nodes_c_dqdot = 6217

snake_nodes_fdaba_dqdot = 11002

snake_nodes_fdcrba_dqdot = 27472


snake_nodes_id_dqddot= 2465

snake_nodes_fdaba_dtau = 4991

snake_nodes_fdcrba_dtau = 13636


In [19]:

snake_nodes_id= 710

snake_nodes_c= 572

snake_nodes_m= 1217

snake_nodes_g= 224

snake_nodes_fdaba 2272

snake_nodes_fdcrba= 14786


snake_nodes_id_dq= 7128

snake_nodes_c_dq= 6217

snake_nodes_m_dq = 11635

snake_nodes_g_dq = 2347

snake_nodes_fdaba_dq = 30719

snake_nodes_fdcrba_dq = 417474


snake_nodes_id_dqdot= 5755

snake_nodes_c_dqdot = 6217

snake_nodes_fdaba_dqdot = 11002

snake_nodes_fdcrba_dqdot = 27472


snake_nodes_id_dqddot= 2465

snake_nodes_fdaba_dtau = 4991

snake_nodes_fdcrba_dtau = 13636


SyntaxError: invalid syntax (<ipython-input-19-11f284279990>, line 10)